# Practice

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('/content/insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


# Splitting dataset into train and test datasets

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(df[['age','affordibility']],df['bought_insurance'],test_size=0.2,random_state=2)

In [4]:
len(X_train)

22

# Scaling Data

In [5]:
X_train_scaled=X_train.copy()
X_train_scaled['age']=X_train_scaled['age']/100

X_test_scaled=X_test.copy()
X_test_scaled['age']=X_test_scaled['age']/100

In [6]:
X_train_scaled

,age,affordibility
16,0.25,0
6,0.55,0
3,0.52,0
21,0.26,0
26,0.23,1
25,0.54,1
12,0.27,0
4,0.46,1
10,0.18,1
5,0.56,1


# Creating Neural Network Model

In [33]:
#@title
model=keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_scaled, y_train, epochs=5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 13ms/step - loss: 0.5436 - accuracy: 0.7273
Epoch 2502/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5436 - accuracy: 0.7273
Epoch 2503/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5435 - accuracy: 0.7273
Epoch 2504/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5435 - accuracy: 0.7273
Epoch 2505/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5435 - accuracy: 0.7273
Epoch 2506/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5434 - accuracy: 0.7273
Epoch 2507/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5434 - accuracy: 0.7273
Epoch 2508/5000
1/1 [==============================] - 0s 13ms/step - loss: 0.5433 - accuracy: 0.7273
Epoch 2509/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5433 - accuracy: 0.7273
Epoch 2510/5000
1/1 [=============================

In [8]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 129ms/step - loss: 0.3153 - accuracy: 1.0000


[0.31525877118110657, 1.0]

# Testing the accuracy of model

In [10]:
X_test_scaled

,age,affordibility
1,0.25,0
0,0.22,1
14,0.49,1
9,0.61,1
19,0.18,1
17,0.58,1


In [9]:
model.predict(X_test_scaled)

array([[0.16282055],
       [0.41183794],
       [0.7255057 ],
       [0.826683  ],
       [0.3651317 ],
       [0.8045075 ]], dtype=float32)

Any value above 0.5 is considered 1 and below 0.5 is considered 0

In [11]:
y_test

1     0
0     0
14    1
9     1
19    0
17    1
Name: bought_insurance, dtype: int64

# Finding the weights and bias used in the model

In [12]:
coef,intercept=model.get_weights()

In [13]:
coef,intercept

(array([[4.9196706],
        [1.4286072]], dtype=float32), array([-2.8673074], dtype=float32))

# Creating the neural network manually

In [14]:
def sigmoid(x):
  import math
  return 1/(1+math.exp(-x))

In [15]:
def prediction_function(age,affordibility):
  weighted_sum=coef[0]*age+coef[1]*affordibility + intercept
  return sigmoid(weighted_sum)

In [16]:
prediction_function(0.49,1)

0.7255056877367211

In [17]:
prediction_function(0.18,1)

0.36513169769602194

Values are same as derived above

# Deriving weights and bias manually

In [18]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [19]:
def sigmoid_numpy(x):
  return 1/(1+np.exp(-x))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [29]:
def gradient_descent(age,affordability,y_true,epochs,loss_thresold):
  w1=w2=0
  bias=0
  rate=0.5
  n=len(age)

  for i in range(epochs):
    weighted_sum=w1*age+w2*affordability+bias
    y_predicted=sigmoid_numpy(weighted_sum)

    loss=log_loss(y_true,y_predicted)

    w1d=(1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
    w2d=(1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))

    bias_d=np.mean(y_predicted-y_true)

    w1=w1-rate*w1d
    w2=w2-rate*w2d
    bias=bias-rate*bias_d

    print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
    if loss<=loss_thresold:
      break
  return w1,w2,bias


In [32]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000,0.4781)

Epoch:0, w1:0.022272727272727274, w2:0.045454545454545456, bias:0.0, loss:0.6931471805599453
Epoch:1, w1:0.042511810594193136, w2:0.08654658432108123, bias:-0.004733293701188009, loss:0.6882664215102942
Epoch:2, w1:0.06114982382351894, w2:0.12407236771195361, bias:-0.013154620695256091, loss:0.684186800721629
Epoch:3, w1:0.07853046882197026, w2:0.15866762748450516, bias:-0.024433961280766563, loss:0.6806432205337531
Epoch:4, w1:0.09492609489398456, w2:0.19083885369445852, bias:-0.03791398951736821, loss:0.6774671235936048
Epoch:5, w1:0.11055222226030463, w2:0.22098945299856623, bias:-0.053074878459116306, loss:0.6745510776173717
Epoch:6, w1:0.12557929337857598, w2:0.24944100912900888, bias:-0.06950585574469893, loss:0.6718262500547012
Epoch:7, w1:0.1401420594861221, w2:0.2764503003136328, bias:-0.08688249613918739, loss:0.6692481715743543
Epoch:8, w1:0.15434702875891984, w2:0.3022228115827399, bias:-0.10494870743290927, loss:0.6667877638928958
Epoch:9, w1:0.16827835811375397, w2:0.3269

(4.9027332785517395, 1.4766264541380267, -2.9095470745147614)

In [31]:
coef,intercept

(array([[4.9196706],
        [1.4286072]], dtype=float32), array([-2.8673074], dtype=float32))

The weights derived manually and by the neural network model are almost the same👍